In [21]:
from google.colab import drive
drive.mount('/content/drive')
!pip install fasttext
import pandas as pd
import fasttext
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Считываем данные и преобразуем под формат fasttext

In [22]:
# считываем данные из ноутбука "X5 tematic model Логика + k-means"
data = pd.read_csv('/content/drive/MyDrive/cleaned_predicted_class_rus.csv')
# считываем выбросы из ноутбука "X5 tematic model Логика + k-means"
outliers = pd.read_csv('/content/drive/MyDrive/outliers_RUS.csv')
# считываем логически размеченные данные из ноутбука "X5 tematic model Логика + k-means"
df_logic = pd.read_csv('/content/drive/MyDrive/df_topics_logic.csv')

In [23]:
# функция которая помогает почистить классы после kmeans
def classifactions(data):
  if data == 'Проблема с получением':
    return 'Проблема с получением'.lower()
  elif data == 'Соедините с оператором':
    return 'уточнить вопрос'.lower()
  elif data == 'Получатель подтверждает что заберт заказ':
    return 'уточнить вопрос'.lower()
  elif data == 'Срок доставки':
    return 'узнать статус посылки'.lower()
  elif data == 'Посылка не пришла':
    return 'Посылка не пришла'.lower()
  elif data == 'На кассе':
    return 'На кассе'.lower()
  elif data == 'Получить информацию о заказе':
    return 'узнать статус посылки'.lower()
  elif data == 'Отказаться от заказа':
    return 'уточнить вопрос'.lower()
  elif data == 'Проблема на пункте выдачи':
    return 'Проблема на пункте выдачи'.lower()
  elif data == 'Местонахождение заказа':
    return 'узнать статус посылки'.lower()
  elif data == 'Узнать дату получения':
    return 'Узнать дату получения или адрес доставки'.lower()
  elif data == 'Заказ пришл':
    return 'Посылка пришла не на тот адрес или не дошла'.lower()
  elif data == 'Не работает постамат':
    return 'не открывается ячейка или не работает постамат'.lower()
  elif data == 'Проблемы с содержимым':
    return 'брак возврат или повреждение'.lower()
  elif data == 'Получить информацию о доставке':
    return 'узнать статус посылки'.lower()
  elif data == 'Узнать срок хранения':
    return 'Узнать срок хранения истек срок хранения'.lower()
  elif data == 'Заказ пришл не по тому адресу':
    return 'Заказ пришел но возникли проблемы с получением'.lower()
  elif data == 'Проблема с открытием ячейки':
    return 'не открывается ячейка или не работает постамат'.lower()
  elif data == 'Номер заказа':
    return 'узнать статус посылки'.lower()
  elif data == 'Как забрать заказ':
    return 'Проблема с получением'.lower()
  elif data == 'Как забрать заказ':
    return 'Проблема с получением'.lower()
  elif data == 'Мне нужно взять посылку из постамата':
    return 'Проблема с получением'.lower()
  elif data == 'СМС для получения заказа':
    return 'уточнить вопрос'.lower()
  else:
    return 'уточнить вопрос'.lower()

data['topic'] = data.predicted_class.apply(lambda x: classifactions(x))
outliers['topic'] = 'связать с оператором'

In [76]:
# объединяем логику, kmeans и выбросы
df = pd.concat([df_logic[['text','topic']], data[['text','topic']]], ignore_index=True)
df = pd.concat([df[['text','topic']], outliers[['text','topic']]], ignore_index=True)
df.head()

,text,topic
0,потеряли код получения заказа,узнать код открытия ячейки
1,мне не пришел код для получения посылки,узнать код открытия ячейки
2,связаться с оператором,связать с оператором
3,я хочу узнать код получения посылки,узнать код открытия ячейки
4,оператора можно связаться,связать с оператором


In [77]:
# частота классов в данных
df.topic.value_counts()

topic
узнать код открытия ячейки                        391234
связать с оператором                              367547
узнать статус посылки                             264543
проблема с получением                             117750
не открывается ячейка или не работает постамат    115142
уточнить вопрос                                    75070
продлить срок хранения заказа                      25015
посылка не пришла                                  10576
вернуть заказ                                       9414
на кассе                                            9360
заказ потерян                                       8422
инвентаризация                                      7834
изменить адрес доставки заказа                      7742
проблема на пункте выдачи                           6851
узнать дату получения или адрес доставки            4342
посылка пришла не на тот адрес или не дошла         3657
изъятие посылки                                     3306
брак возврат или поврежде

In [78]:
tp = df.topic.unique()

In [83]:
# переводим классы в формат чисел
def classifactions(data):
  if data == tp[0]:
    return 1
  elif data == tp[1]:
    return 2
  elif data == tp[2]:
    return 3
  elif data == tp[3]:
    return 4
  elif data == tp[4]:
    return 5
  elif data == tp[5]:
    return 6
  elif data == tp[6]:
    return 7
  elif data == tp[7]:
    return 8
  elif data == tp[8]:
    return 9
  elif data == tp[9]:
    return 10
  elif data == tp[10]:
    return 11
  elif data == tp[11]:
    return 12
  elif data == tp[12]:
    return 13
  elif data == tp[13]:
    return 14
  elif data == tp[14]:
    return 15
  elif data == tp[15]:
    return 16
  elif data == tp[16]:
    return 17
  elif data == tp[17]:
    return 18
  elif data == tp[18]:
    return 19
  elif data == tp[19]:
    return 20

df['topic_num'] = df.topic.apply(lambda x: classifactions(x))

In [87]:
# приводим классы к формату модели
df['class'] = '__label__' + df['topic_num'].astype(str)
df.head(5)

df['category_description'] = df['class'] + ' ' + df['text']
df.head(3)

,text,topic,topic_num,class,category_description
0,потеряли код получения заказа,узнать код открытия ячейки,1,__label__1,__label__1 потеряли код получения заказа
1,мне не пришел код для получения посылки,узнать код открытия ячейки,1,__label__1,__label__1 мне не пришел код для получения пос...
2,связаться с оператором,связать с оператором,2,__label__2,__label__2 связаться с оператором


In [88]:
# делим данные на обучающую и тестовую выборки
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [89]:
train.to_csv("/content/drive/MyDrive/x5_data_train", columns=["category_description"], index=False, header=False)
test.to_csv("/content/drive/MyDrive/x5_data_test", columns=["category_description"], index=False, header=False)

# Обучаем модель

In [90]:
import fasttext
from sklearn.metrics import accuracy_score

model = fasttext.train_supervised(input="/content/drive/MyDrive/x5_data_train")

# прверяем качество модели (precision, recall)
result = model.test("/content/drive/MyDrive/x5_data_test")
result

(287049, 0.9574637082867385, 0.9574637082867385)

In [91]:
# Extract evaluation metrics from the result
precision = result[1]
recall = result[2]
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Calculate accuracy as the average of precision, recall, and F1-score
accuracy = (precision + recall + f1_score) / 3

print("Accuracy:", accuracy)

Accuracy: 0.9574637082867387


# Делаем предсказание

In [92]:
model.predict("связать с оператором")

(('__label__2',), array([1.00001001]))